In [1]:
import os
from string import punctuation
from pymystem3 import Mystem
import operator
from stop_words import get_stop_words
import json

#Функция для извлечения ключевых биграмм из текста

In [2]:
def my_analyzer(file):
    m = Mystem()
    with open(file, 'r', encoding="UTF-8") as f:
        text = f.read()
    non_lemmatized = [token.strip(punctuation) for token in text.split() if token.isalnum()]
    text_string = ' '.join(non_lemmatized)
    lemmatized = m.lemmatize(text_string) #лемматизируем текст
    lemmas = []
    for lemma in lemmatized:
        if lemma.isalnum():
            lemmas.append(lemma) #избавляемся от остатков пунктуации
    bigrams = [] #создаем список биграмм (из лемм)
    for i in range(len(lemmas) - 1):
        bigram = "{} {}".format(lemmas[i], lemmas[i+1])
        bigrams.append(bigram)
    bigram_dict = {} #считаем повторяющиеся биграммы и складываем в словарь
    for bigram in bigrams:
        n = bigrams.count(bigram)
        bigram_dict[bigram] = n
    sorted_dict = {}
    stop_words = get_stop_words('ru')
    for key, value in bigram_dict.items():
        if key.split()[0] in stop_words or key.split()[1] in stop_words: #если хотя бы одно слово в биграмме есть в списке стоп-слов, то выбрасываем ее
            pass
        else:
            sorted_dict[key] = value #если оба слова в биграмме не стоп-слова, то складываем их в новый словарь
    sorted_counts = sorted(sorted_dict.items(), key=operator.itemgetter(1), reverse=True) #сортируем
    number = dict(sorted_counts[:5]) #делаем срез из 5 самых частотных
    print(number)

#Функция для подсчета частотности лемм

In [4]:
def lemma_counter(file):
    m = Mystem()
    with open(file, 'r', encoding="UTF-8") as f:
        text = f.read()
    non_lemmatized = [token.strip(punctuation) for token in text.split() if token.isalnum()]
    text_string = ' '.join(non_lemmatized)
    lemmatized = m.lemmatize(text_string) #лемматизируем текст
    lemmas = []
    for lemma in lemmatized:
        if lemma.isalnum():
            lemmas.append(lemma) #избавляемся от остатков пунктуации
    lemmas_dict = {}
    for lemma in lemmas:
        n = lemmas.count(lemma) #считаем частотность лемм в тексте
        lemmas_dict[lemma] = n #и складываем их в словарь
    sorted_dict = {}
    stop_words = get_stop_words('ru')
    for key, value in lemmas_dict.items():
        if key in stop_words: #если лемма - это стоп-слово, то выкидываем ее
            pass
        else:
            sorted_dict[key] = value #а если нет - то добавляем в словарь
    sorted_counts = sorted(sorted_dict.items(), key=operator.itemgetter(1), reverse=True) #сортируем
    number = dict(sorted_counts[:5]) #делаем срез из 5 самых частотных
    print(number)

#Тестируем на случайных текстах

In [ ]:
os.chdir('sample_data')

In [9]:
for file in os.listdir():
    print(os.path.basename(file)) #печатаем имя файла
    lemma_counter(file) #печатаем самые частотные леммы
    my_analyzer(file) #печатаем самые частотные биграммы

2.txt
{'картина': 6, 'описание': 4, 'левитан': 3, 'сюжет': 3, 'любовь': 2}
{'описание картина': 4, 'картина левитан': 2, 'автор использовать': 2, 'художник передвижник': 1, 'передвижник описание': 1}
3.txt
{'маленький': 3, 'белый': 3, 'картина': 2, 'корзухин': 2, 'комнатка': 2}
{'описание картина': 1, 'картина корзухин': 1, 'корзухин описывать': 1, 'зритель представать': 1, 'представать маленький': 1}
txt.txt
{'картина': 22, 'свой': 8, 'изображать': 8, 'художник': 7, 'левитан': 7}
{'описание картина': 5, 'картина левитан': 3, 'суриков описание': 2, 'картина европа': 2, 'европа суриков': 2}


#Тестируем на датасете russia_today_0.jsonlines

In [23]:
data = []
with open("russia_today_0.jsonlines", "r", encoding="UTF-8") as read_file: #откроем файл и построчно считаем данные
    for line in read_file:
        data.append(json.loads(line))

In [24]:
os.mkdir('data')
os.chdir('data')

In [25]:
i = 1
for item in data:
    with open('{}.txt'.format(i), 'w', encoding="UTF-8") as fl: #запишем каждый текст в отдельный txt файл
        fl.write(item['content'])
    i += 1

In [27]:
for file in os.listdir()[:20]: #делаем срез из 20 файлов
    print(os.path.basename(file)) #печатаем имя файла
    lemma_counter(file) #печатаем самые частотные леммы
    my_analyzer(file) #печатаем самые частотные биграммы

1.txt
{'турция': 23, 'турецкий': 17, 'сша': 12, 'американский': 11, 'санкция': 10}
{'американский санкция': 3, 'вводить санкция': 3, 'фонд защита': 3, 'защита демократия': 3, 'президент турция': 2}
10.txt
{'власть': 11, 'мьянма': 9, 'гуманитарный': 8, 'мусульманин': 8, 'рохинджа': 6}
{'бирманский власть': 4, 'прекращение огонь': 3, 'власть мьянма': 3, 'официальный представитель': 2, 'аун сан': 2}
100.txt
{'турнир': 20, 'сет': 14, 'победа': 13, 'четвертьфинал': 13, 'пробиваться': 12}
{'турнир большой': 7, 'Australian Open': 6, 'ракетка планета': 5, 'январь 2018': 5, '22 январь': 4}
101.txt
{'музей': 11, 'российский': 5, 'институт': 5, 'археология': 5, 'малый': 4}
{'институт археология': 5, 'малый николаевский': 4, 'николаевский дворец': 4, 'чудово монастырь': 4, 'археология российский': 4}
102.txt
{'трамп': 10, 'победа': 10, 'свой': 8, 'сша': 8, 'дональд': 7}
{'дональд трамп': 6, 'южный корея': 2, '3 январь': 2, 'ким чен': 2, 'соединять штат': 2}
103.txt
{'ядерный': 15, 'корея': 13, 'ис